In [19]:
import selenium
import os
import time
import io
from PIL import Image
import hashlib
import requests

In [20]:
DRIVER_PATH = "C:/Users/andre/Desktop/Image_Scraper_Generator/chromedriver.exe"

In [21]:
#Instantiate a webdriver object
from selenium import webdriver
wd = webdriver.Chrome(executable_path = DRIVER_PATH)

In [22]:
# Sort dialog box.
#active the iframe and click the agree button
#https://stackoverflow.com/questions/65286557/python-selenium-click-google-i-agree-button

In [23]:
# Function to get image urls from query, n links, and webdriver object
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [24]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [25]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [26]:
# Perform operation
search_term = "Chihuahua"

search_and_download(
    search_term = search_term,
    driver_path=DRIVER_PATH,
    number_images = 10)

Found: 100 search results. Extracting links from 0:100
Found: 10 image links, done!
SUCCESS - saved https://www.thesprucepets.com/thmb/Kh-xt27-hqeQzhyr9288cl_P64I=/1396x1396/smart/filters:no_upscale()/twenty20_f84c633e-705e-4bf8-a724-00cdea750d8d-590b51893df78c92837b18d6.jpg - as ./images\chihuahua\efbee4890f.jpg
SUCCESS - saved https://s36700.pcdn.co/wp-content/uploads/2015/05/shutterstock_1741426311.jpg.optimal.jpg - as ./images\chihuahua\fea3307749.jpg
SUCCESS - saved https://img.pixers.pics/pho_wat(s3:700/FO/14/08/96/0/700_FO1408960_12d7da573b645ef8371da12106eb46ea.jpg,700,700,cms:2018/10/5bd1b6b8d04b8_220x50-watermark.png,over,480,650,jpg)/wall-murals-chihuahua.jpg.jpg - as ./images\chihuahua\6be3ce58f1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTXOxbpXiBjD_itMa0YS7bLcC_rb2Zx-zTHaw&usqp=CAU - as ./images\chihuahua\1179890d15.jpg
SUCCESS - saved https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/27134512/Chihuahua-at-the-AKC-Na